# EOD historical data API extraction

With the suscription, information about the fundamentals, prices, etc will be retrived from the API. That data will be stored in csv format locally.

In [ ]:
!pip install eod
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing libraries
import os
import config as cfg
import urllib, json
import requests
import pandas as pd
from io import StringIO
import datetime
import eod
from eod import EodHistoricalData
import requests
import ssl
import numpy as np

First, try with the demo:

In [ ]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://eodhistoricaldata.com/api/technical/AAPL.US?order=d&fmt=json&from=1900-08-01&to=2022-31-12&function=sma&period=50&api_token=demo'
data = urllib.request.urlopen(url, context=ctx).read().decode()

print(data)
print("Retrieved", len(data), "characters")

#STR to XML tree
prices = json.loads(data)
s=0

prices_formatted_str = json.dumps(prices, indent=2)

[]
Retrieved 2 characters


## 1. Get all the US tickers available

In [ ]:
import urllib.request, urllib.parse, urllib.error #For make easier the socket connection
from bs4 import BeautifulSoup #Livery for html fixing errors
import ssl #For secure connection
import re

This code retrieves all the tickers for the US market, more than 50,000 tickers are retrieved:

In [ ]:
api_key = 'AAAAAAAAAAA'

# Markets to obtain tickers:
markets = ['US']
tickers2 = {}

for market in markets:
    # Define the API endpoint to retrieve the list of exchanges
    endpoint = 'https://eodhistoricaldata.com/api/exchange-symbol-list/{}?api_token={}'.format(market,api_key)

    # Send a GET request to the API endpoint
    response = requests.get(endpoint)
    # Extract the data from the response JSON object
    text = response.text


In [ ]:
# Convert the string to a list of lines
csv_lines = text.split('\n')

# Remove the empty last line if present
if csv_lines[-1] == '':
    csv_lines = csv_lines[:-1]

# Convert the lines to a list of lists (rows)
csv_rows = [line.split(',') for line in csv_lines]
csv_rows = [line for line in csv_rows if len(line)==7]

# Create a pandas dataframe from the list of rows
df2 = pd.DataFrame(csv_rows[1:], columns=csv_rows[0])

# Only select the "Common stocks" type which are traded at NYSE or NASDAQ:
df2 = df2[(df2['Type']=='"Common Stock"') & ((df2['Exchange']=='NASDAQ') | (df2['Exchange']=='NYSE'))]


The obtained companies are below:

In [ ]:
df2

,Code,Name,Country,Exchange,Currency,Type,Isin
0,A,"""Agilent Technologies Inc""",USA,NYSE,USD,"""Common Stock""",US00846U1016
1,AA,"""Alcoa Corp""",USA,NYSE,USD,"""Common Stock""",US0138721065
44,AAC,"""AAC Holdings Inc""",USA,NYSE,USD,"""Common Stock""",KYG330321061
45,AAC-UN,"""Ares Acquisition Corp""",USA,NASDAQ,USD,"""Common Stock""",
46,AAC-WT,"""Ares Acquisition Corporation Re""",USA,NYSE,USD,"""Common Stock""",
...,...,...,...,...,...,...,...
50400,ZWZZT,"""NASDAQ TEST STOCK""",USA,NASDAQ,USD,"""Common Stock""",
50401,ZXYZ-A,"""NASDAQ SYMBOLOGY TEST""",USA,NASDAQ,USD,"""Common Stock""",
50402,ZYME,"""Zymeworks Inc. Common Stock""",USA,NASDAQ,USD,"""Common Stock""",CA98985W1023
50403,ZYNE,"""Zynerba Pharmaceuticals Inc""",USA,NASDAQ,USD,"""Common Stock""",US98986X1090


Cantidad de empresas obtenidas:

In [ ]:
print(len(df2))

7297


In [ ]:
# We can print one line, of Apple, for example!
print(df2[df2['Code']=='AAPL'])

     Code         Name Country Exchange Currency            Type          Isin
202  AAPL  "Apple Inc"     USA   NASDAQ      USD  "Common Stock"  US0378331005


In [ ]:
print(type(df2['Code']))

<class 'pandas.core.series.Series'>


## 2. Get all the information for one given ticker

In [ ]:
tickers = df2['Code'].tolist()

In [ ]:
print(tickers)

['A', 'AA', 'AAC', 'AAC-UN', 'AAC-WT', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACT-U', 'AADI', 'AAIC', 'AAIC-P-B', 'AAIC-P-C', 'AAIN', 'AAL', 'AAM-P-A', 'AAM-P-B', 'AAM-PA', 'AAM-PB', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAT', 'AATC', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABM', 'ABNB', 'ABOS', 'ABR', 'ABR-PD', 'ABR-PE', 'ABR-PF', 'ABSI', 'ABST', 'ABT', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAB', 'ACABU', 'ACABW', 'ACAC', 'ACACU', 'ACACW', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAQ-UN', 'ACAQ-WT', 'ACAX', 'ACAXR', 'ACAXU', 'ACAXW', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACCD', 'ACCO', 'ACDC', 'ACDCW', 'ACEL', 'ACER', 'ACET', 'ACGL', 'ACGN', 'ACHC', 'ACHL', 'ACHR', 'ACHV', 'ACI', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACONW', 'ACOR', 'ACP-PA', 'ACR', 'ACR-PD', 'ACRE', 'ACRO', 'ACRO-WT', 'ACRS', 'ACRV', 'ACRX', 'ACST', 'ACT', 'ACTG', 'ACVA', 'ACXP', 'ADAG', 'ADAL', 'ADALU', 'ADALW', 'ADAP', 'ADBE', 'ADC', 'ADC-PA

This code generates a list of elements which contains for each ticker, the whole fundamental data of the company. That list willbe processes after into a dataframe. It will take a long time to run, and it will not be able to run entirly in one day (because of the maximum requests available per user 100,000).

In [ ]:
# Set the api key
api_key = 'AAAAAAAAAAA'

whole_data = []
prices = []

# Process the loop in stages for avoiding RAM collapse
for ticker in tickers[6000:]:
    ## FUNDAMENTALS:
    # Define the API endpoint and the ticker symbol
    endpoint1 = 'https://eodhistoricaldata.com/api/fundamentals/{}.US?api_token={}'.format(ticker,api_key)

    # Send a GET request to the API endpoint
    response1 = requests.get(endpoint1)

    # Check if the request was successful
    if response1.status_code == 200:
        # Extract the data from the response JSON object
        data1 = response1.json()
        # Append the data into the list
        whole_data.append(data1)

    ## PRICES:

    endpoint2 = 'https://eodhistoricaldata.com/api/eod/{}.US?from=1900-01-01&to=2023-01-01&period=d&fmt=json&api_token={}'.format(ticker,api_key)
    response2 = requests.get(endpoint2)

    # Check if the request was successful
    if response2.status_code == 200:
        # Extract the data from the response JSON object
        data2 = response2.json()
        # Append the data into the list
        prices.append(data2)

    else:
        print('Request failed with status code: {}'.format(response.status_code))

    if len(whole_data) % 200 == 0:
        print('Tickers aquired: ', len(whole_data))

Tickers aquired:  200
Tickers aquired:  400
Tickers aquired:  600
Tickers aquired:  800
Tickers aquired:  1000
Tickers aquired:  1200


Print some numbers:

In [ ]:
print(len(whole_data))      # Length of the accured data (number of companies)
print(len(prices))          # Length of the prices aquired
print(len(tickers))         # Number of companies available

1297
1297
7297


In [ ]:
print(whole_data[3])  # Print the 100th company

{'General': {'Code': 'SLGCW', 'Type': 'Common Stock', 'Name': 'SomaLogic Inc', 'Exchange': 'NASDAQ', 'CurrencyCode': 'USD', 'CurrencyName': 'US Dollar', 'CurrencySymbol': '$', 'CountryName': 'USA', 'CountryISO': 'US', 'OpenFigi': None, 'ISIN': None, 'LEI': None, 'PrimaryTicker': None, 'CUSIP': None, 'CIK': '1837412', 'EmployerIdNumber': None, 'FiscalYearEnd': 'December', 'IPODate': '2021-04-16', 'InternationalDomestic': None, 'Sector': 'Healthcare', 'Industry': 'Health Information Services', 'GicSector': None, 'GicGroup': None, 'GicIndustry': None, 'GicSubIndustry': None, 'HomeCategory': 'Warrant', 'IsDelisted': False, 'Description': "SomaLogic, Inc. operates as a protein biomarker discovery and clinical diagnostics company in the United States. It develops slow off-rate modified aptamers (SOMAmers), which are modified nucleic acid-based protein binding reagents that are specific for its cognate protein; and offers proprietary SomaScan services, which provide multiplex protein detectio

Converge all *json* information into a *pandas* data frame.

In [ ]:
dataframe_list = []

for i in range(len(whole_data)):
    try:
        fundamentals = whole_data[i]

        # Outstanding shares for a given company:
        shares_df = pd.DataFrame.from_dict(fundamentals['outstandingShares']['annual'],orient='index')
        # Some preprocessing:
        shares_df.drop(columns='date',inplace=True)
        shares_df.rename(columns={'dateFormatted': 'date'}, inplace=True)
        shares_df

        # Other data frames extracted for each company:
        balance_df = pd.DataFrame.from_dict(fundamentals['Financials']["Balance_Sheet"]['yearly'],orient='index')
        cashflow_df = pd.DataFrame.from_dict(fundamentals['Financials']['Cash_Flow']['yearly'],orient='index')
        income_df = pd.DataFrame.from_dict(fundamentals['Financials']["Income_Statement"]['yearly'],orient='index')

        # Merge into a fundamentas dataframe:
        df_global = balance_df.merge(shares_df, how='inner', on='date')
        df_global = df_global.merge(cashflow_df, how='inner', on='date')
        df_global = df_global.merge(income_df, how='inner', on='date')

        # Some constant columns to identify the company and the sector:
        df_global = df_global.assign(Ticker=whole_data[i]['General']['Code'],
                                    Industry=fundamentals['General']['GicIndustry'],
                                    Sector=fundamentals['General']['Sector'])

        # Constructing the price for the response variable:
        eod = prices[i]
        eod_df = pd.DataFrame.from_dict(eod)
        eod_df = eod_df[['date','close','adjusted_close']]

        df_global['date_'] = [str(x)[0:7] for x in df_global['date']]           # Generate a column to average at every month
        df_global['price'] = np.nan
        df_global['price_adjusted'] = np.nan
        eod_df['date_'] = [str(x)[0:7] for x in eod_df['date']]                 # Generate a column to average at every month

        for j in range(len(df_global)):
            df_global['price'].iloc[j] = eod_df['close'][eod_df['date_'] == df_global['date_'].iloc[j]].mean()  # Select price as the average at the month
            df_global['price_adjusted'].iloc[j] = eod_df['adjusted_close'][eod_df['date_'] == df_global['date_'].iloc[j]].mean()  # Select price as the average at the month


    except:
        pass

    dataframe_list.append(df_global)

Se han truncado las últimas 5000 líneas del flujo de salida.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_global['price'].iloc[j] = eod_df['close'][eod_df['date_'] == df_global['date_'].iloc[j]].mean()  # Select price as the average at the month
<ipython-input-17-7c76266d3576>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_global['price_adjusted'].iloc[j] = eod_df['adjusted_close'][eod_df['date_'] == df_global['date_'].iloc[j]].mean()  # Select price as the average at the month
<ipython-input-17-7c76266d3576>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

Calculate the key fundamental ratios:

In [ ]:
for df_global in dataframe_list:

    try:
        # Adding the fundamental ratios:
        df_global['eps'] = df_global['netIncome_x'].astype(float)/df_global['commonStockSharesOutstanding'].astype(float)
    except:
        pass

    try:
        df_global['per'] = df_global['price']*df_global['commonStockSharesOutstanding'].astype(float)/df_global['netIncome_x'].astype(float)
    except:
        pass

    try:
        df_global['der'] = df_global['totalCurrentLiabilities'].astype(float)/df_global['totalStockholderEquity'].astype(float)
    except:
        pass
        df_global['roe'] = df_global['netIncome_x'].astype(float)/df_global['totalStockholderEquity'].astype(float)

    try:
        df_global['wcr'] = df_global['totalCurrentAssets'].astype(float)/df_global['totalCurrentLiabilities'].astype(float)
    except:
        pass

    try:
        df_global['qr'] = (df_global['totalCurrentAssets'].astype(float)-df_global['inventory'].astype(float))/df_global['totalCurrentLiabilities'].astype(float)
    except:
        pass

    try:
        df_global['fcfps'] = df_global['freeCashFlow'].astype(float)/df_global['commonStockSharesOutstanding'].astype(float)
    except:
        pass

    try:
        df_global['pcfr'] = df_global['fcfps']/df_global['price'].astype(float)
    except:
        pass

In [ ]:
dataframe_list[1].columns

Index(['date', 'filing_date_x', 'currency_symbol_x', 'totalAssets',
       'intangibleAssets', 'earningAssets', 'otherCurrentAssets', 'totalLiab',
       'totalStockholderEquity', 'deferredLongTermLiab',
       ...
       'date_', 'price', 'price_adjusted', 'eps', 'per', 'der', 'wcr', 'qr',
       'fcfps', 'pcfr'],
      dtype='object', length=143)

Feature ordering:

In [ ]:
features = ['date','Ticker','Industry','Sector',                                # Company general
            'eps','per','der','wcr','qr', 'fcfps', 'pcfr',                       # Fundamental ratios
            'totalAssets', 'intangibleAssets', 'otherCurrentAssets',            # Assets
            'totalLiab','otherCurrentLiab',                                     # Liabilities
            'beginPeriodCashFlow','endPeriodCashFlow', 'totalCashFromOperatingActivities',                      # Cash
            'changeInCash','otherCashflowsFromInvestingActivities', 'changeInWorkingCapital', 'freeCashFlow',   # Cash
            'researchDevelopment', 'incomeBeforeTax', 'netIncome_x', 'sellingGeneralAdministrative', 'grossProfit',  # Balance
            'ebit','ebitda','depreciationAndAmortization', 'operatingIncome', 'totalOperatingExpenses',              # Balance
            'taxProvision', 'totalRevenue', 'costOfRevenue',                                                         # Balance
            'dividendsPaid',                                                    # Dividends
            'commonStockSharesOutstanding',                                     # Total shares outstanding
            'price', 'price_adjusted'
            ]

dataset = pd.DataFrame(columns=features)

for df in dataframe_list:

    if len(df) > 0:
        try:
            dataset = pd.concat([dataset,df[features]])
        except:
            pass

The obtained dataset is the following:

In [ ]:
dataset

,date,Ticker,Industry,Sector,eps,per,der,wcr,qr,fcfps,...,depreciationAndAmortization,operatingIncome,totalOperatingExpenses,taxProvision,totalRevenue,costOfRevenue,dividendsPaid,commonStockSharesOutstanding,price,price_adjusted
0,2022-12-31,SLF,Insurance,Financial Services,6.705983,6.916638,0.181566,2.988363,NaN,7.369231,...,528000000.00,4312000000.00,19285000000.00,713000000.00,23207000000.00,None,1671000000.00,585000000.00,46.382857,45.882105
1,2021-12-31,SLF,Insurance,Financial Services,7.470085,7.268871,0.092726,95.034426,NaN,-3.312821,...,438000000.00,5424000000.00,30658000000.00,727000000.00,35755000000.00,None,1428000000.00,585000000.00,54.299091,51.375500
2,2020-12-31,SLF,Insurance,Financial Services,5.580645,7.922267,0.113695,6.546370,NaN,12.045840,...,406000000.00,3631000000.00,40050000000.00,495000000.00,43337000000.00,None,1360000000.00,589000000.00,44.211364,40.397173
3,2019-12-31,SLF,Insurance,Financial Services,5.499761,8.181134,0.082614,7.444904,7.444904,4.138855,...,378000000.00,3562000000.00,27466000000.00,284000000.00,39679000000.00,2272107891.98,1318000000.00,587843688.00,44.994286,39.341257
4,2018-12-31,SLF,Insurance,Financial Services,4.769231,7.025739,0.090652,85.030247,85.030247,6.135843,...,226000000.00,3816000000.00,18119000000.00,597000000.00,26997000000.00,839792796.16,1227000000.00,611000000.00,33.507368,28.187863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2002-12-31,ZYXI,Health Care Equipment & Supplies,Healthcare,-0.637927,-69.365256,-1.000000,NaN,NaN,-1.523440,...,None,-22720.00,22720.00,None,None,None,None,148531.00,44.250000,21.652413
21,2001-12-31,ZYXI,Health Care Equipment & Supplies,Healthcare,-81.968173,-8.539900,-0.984713,NaN,NaN,-20.267982,...,None,-37354.00,37354.00,None,None,None,None,1571.00,700.000000,342.524400
22,2000-12-31,ZYXI,Health Care Equipment & Supplies,Healthcare,-1042.931366,-13.843193,-1.220816,0.038355,0.001797,-683.431046,...,13621.00,-1598223.00,1526377.00,None,160883.00,232369.00,None,1559.00,14437.499800,7064.566520
23,1999-12-31,ZYXI,Health Care Equipment & Supplies,Healthcare,-2.101061,NaN,NaN,NaN,NaN,NaN,...,366.00,-11342.00,17549.00,None,12414.00,6207.00,None,8015.00,NaN,NaN


In [ ]:
dataset.to_csv('dataset-01-05_part3.csv', sep=';')